# Regression


## Introduction

In this chapter, you'll learn about to do regressions with code.

If you're running this code (either by copying and pasting it, or by downloading it using the icons at the top of the page), you may need to the packages it uses by, for example, running `pip install packagename` on your computer's command line. (If you're not sure what a command line is, take a quick look at the basics of coding chapter.)

Most of this chapter will rely on [statsmodels](https://www.statsmodels.org/stable/index.html).

Some of the material in this chapter follows [Grant McDermott](https://grantmcdermott.com/)'s excellent notes and the [Library of Statistical Translation](https://lost-stats.github.io/).

### Notation and basic definitions

Greek letters, like $\beta$, are the truth and represent parameters. Modified Greek letters are an estimate of the truth, for example $\hat{\beta}$. Sometimes Greek letters will stand in for vectors of parameters. Most of the time, upper case Latin characters such as $X$ will represent random variables (which could have more than one dimension). Lower case letters from the Latin alphabet denote realised data, for instance $x$ (which again could be multi-dimensional).  Modified Latin alphabet letters denote computations performed on data, for instance $\bar{x} = \frac{1}{n} \displaystyle\sum_{i} x_i$ where $n$ is number of samples.

Ordinary least squares (OLS) regression is used to predict the value of an outcome variable $y$ based on one or more input predictor variables arrange in a matrix $X$. The equation that we would like to recover is

$$
y = \beta_0 + \beta_1 \cdot x_1 + \beta_2 \cdot x_2
$$

where the $x_i$ could be transforms of original data. But this is a platonic ideal, what is called the data generating process (DGP). All we can do is recover *estimates* of it, i.e. we can find $\hat{\beta_i}$ and the relationship

$$
y = \hat{\beta_0} + \hat{\beta_1} \cdot x_1 + \hat{\beta_2} \cdot x_2 + \epsilon
$$

This equation can also be expressed in matrix form as

$$
y = x'\cdot \hat{\beta} + \epsilon
$$

where $x' = (1, x_1, \dots, x_{n})'$ and $\hat{\beta} = (\hat{\beta_0}, \hat{\beta_1}, \dots, \hat{\beta_{n}})$.

Given data $y_i$ stacked to make a vector $y$ and $x_{i}$ stacked to make a matrix $X$, this can be solved for the coefficients $\hat{\beta}$ according to

$$
\hat{\beta} = \left(X'X\right)^{-1} X'y
$$

The interpretation of these regression coefficients depends on what their units are to begin with, but you can always work it out by differentiating both sides of the model equation with respect to the $x_i$. For example, for the first model equation above

$$
\frac{\partial y}{\partial x_i} = \beta_i
$$

so we get the interpretation that $\beta_i$ is the rate of change of y with respect to $x_i$. If $x_i$ and $y$ are in levels, this means that a unit increase in $x_i$ is associated with a $\beta_i$ units increase in $y$. If the right-hand side of the model is $\ln x_i$ then we get

$$
\frac{\partial y}{\partial x_i} = \beta_i \frac{1}{x_i} 
$$

with some abuse of notation, we can rewrite this as $dy = \beta_i dx_i/x_i$, which says that a percent change in $x_i$ is associated with a $\beta_i$ unit change in $y$. With a logged $y$ variable, it's a percent change in $x_i$ that is associated with a percent change in $y$, or $dy/y = \beta_i dx_i/x_i$ (note that both sides of this equation are unitless in this case). Finally, another example that is important in practice is that of log differences, eg $y = \beta_i (\ln x_i - \ln x_i')$. Again, we will abuse notation and say that this case may be represented as $dy = \beta_i (d x_i/x_i - dx_i'/x_i')$, i.e. the difference in two percentages, a *percentage point* change, in $x_i$ is associated with a $\beta_i$ unit change in $y$.

### Imports

Let's import the packages we'll be using:

In [ ]:
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import os

In [ ]:
# TODO hide cell
# Set max rows displayed for readability
pd.set_option('display.max_rows', 6)
# Plot settings
plot_style = {'xtick.labelsize': 20,
                  'ytick.labelsize': 20,
                  'font.size': 22,
                  'figure.autolayout': True,
                  'figure.figsize': (10, 5.5),
                  'axes.titlesize': 22,
                  'axes.labelsize': 20,
                  'lines.linewidth': 4,
                  'lines.markersize': 6,
                  'legend.fontsize': 16,
                  'mathtext.fontset': 'stix',
                  'font.family': 'STIXGeneral',
                  'legend.frameon': False}
plt.style.use(plot_style)

## Regression basics

There are two ways to run regressions in [**statsmodels**](https://www.statsmodels.org/stable/index.html); passing the data directly as objects, and using formulae. We'll see both but, just to get things started, let's use the formula API.

We'll use the starwars dataset to run a regression of mass on height for star wars characters. First, let's bring the dataset in:

In [ ]:
df = pd.read_pickle(os.path.join('data', 'starwars.pickle'))
# Look at first few rows
df.head()

Okay, now let's do a regression using OLS and a formula that says our y-variable is mass and our regressor is height:

In [ ]:
results = smf.ols('mass ~ height', data=df).fit()

Well, where are the results!? They're stored in the object we created. To peek at them we need to call the summary function (and, for easy reading, I'll print it out too using `print`)

In [ ]:
print(results.summary())

What we're seeing here are really several tables glued together. To just grab the coefficients in a tidy format, use

In [ ]:
results.summary().tables[1]

You'll have noticed that we got an intercept, even though we didn't specify one in the formula. **statsmodels** adds in an intercept by default because, most of the time, you will want one. To turn it off, add a `-1` at the end of the formula command, eg in this case you would call `smf.ols('mass ~ height -1', data=df).fit()`.

The fit we got in the case with the intercept was pretty terrible; a low $R^2$ and both of our confidence intervals are large and contain zero. What's going on? If there's one adage in regression that's always worth paying attention to, it's *always plot your data*. Let's see what's going on here:

In [ ]:
fig, ax = plt.subplots()
sns.scatterplot(data=df, x="height", y="mass",
                s=250, ax=ax, legend=False,
                alpha=0.6)
ax.annotate('Jabba the Hutt', df.iloc[df['mass'].idxmax()][['height', 'mass']],
            xytext=(0, -50), textcoords='offset points',
            arrowprops=dict(arrowstyle="fancy",
                            color='k',
                            connectionstyle="arc3,rad=0.3",
                            )
            )
ax.set_title('Always look for outliers!', loc='left')
plt.show()

Oh dear, Jabba's been on the paddy frogs again, and he's a bit of different case. When we're estimating statistical relationships, we have all kinds of choices and should be wary about arbitrary decisions of what to include or exclude in case we fool ourselves about the generality of the relationship we are capturing. Let's say we knew that we weren't interested in Hutts though, but only in other species: in that case, it's fair enough to filter out Jabba and run the regression without this obvious outlier. We'll exclude any entry that contains the string 'Jabba' in the `name` column:

In [ ]:
results_outlier_free = smf.ols('mass ~ height', data=df[~df['name'].str.contains('Jabba')]).fit()
print(results_outlier_free.summary())

This looks a lot more healthy. Not only is the model explaining a *lot* more of the data, but the coefficients are now significant.

### Standard errors

You'll have seen that there's a column for the standard error of the estimates in the regression table and a message saying that the covariance type of these is 'nonrobust'. Let's say that, instead, we want to use Eicker-Huber-White robust standard errors, aka "HC2" standard errors. We can specify to use these up front standard errors up front in the fit method:

In [ ]:
(smf.ols('mass ~ height', data=df)
    .fit(cov_type='HC2')
    .summary()
    .tables[1])

Or, alternatively, we can go back to our existing results and recompute the results from those:

In [ ]:
print(results.get_robustcov_results('HC2').summary())

There are several different types of standard errors available in **statsmodels**:

- ‘HC0’, ‘HC1’, ‘HC2’, and ‘HC3’
- ‘HAC’, for heteroskedasticity and autocorrelation consistent standard errors, for which you may want to also use some keyword arguments
- 'hac-groupsum’, for Driscoll and Kraay heteroscedasticity and
autocorrelation robust standard errors in panel data, again for which you may have to specify extra keyword arguments
- 'hac-panel’, for heteroscedasticity and autocorrelation robust standard
errors in panel data, again with keyword arguments; and
- 'cluster' for clustered standard errors.

You can find information on all of these [here](https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.OLSResults.get_robustcov_results.html?highlight=get_robustcov_results#statsmodels.regression.linear_model.OLSResults.get_robustcov_results). For now, let's look more closely at those last ones: clustered standard errors.



#### Clustered standard errors

Often, we know something about the structure of likely errors, namely that they occur in groups. In the below example we use one-way clusters to capture this effect in the errors.

Note that in the below example, we grab a subset of the data for which a set of variables we're interested in are defined, otherwise the below example would execute with an error because of missing cluster-group values.

In [ ]:
xf = df.dropna(subset=['homeworld', 'mass', 'height', 'species'])
results_clus = (smf.ols('mass ~ height', data=xf)
                   .fit(cov_type='cluster', cov_kwds={'groups': xf['homeworld']}))
print(results_clus.summary())

We can add two-way clustering of standard errors using the following:

In [ ]:
xf = df.dropna(subset=['homeworld', 'mass', 'height', 'species'])
two_way_clusters = np.array(xf[['homeworld', 'species']], dtype=str)
results_clus = (smf.ols('mass ~ height', data=xf)
                   .fit(cov_type='cluster',
                        cov_kwds={'groups': two_way_clusters}))
print(results_clus.summary())

As you would generally expect, the addition of clustering has increased the standard errors.

## Fixed effects and categorical variables

Fixed effects are a way of allowing the intercept of a regression model to vary freely across individuals or groups. It is, for example, used to control for any individual-specific attributes that do not vary across time in panel data.

Let's use the 'mtcars' dataset to demonstrate this. We'll read it in and set the datatypes of some of the columns at the same time.

In [ ]:
df = (pd.read_csv('https://raw.githubusercontent.com/LOST-STATS/lost-stats.github.io/source/Data/mtcars.csv',
                  dtype={'model': str, 'mpg':float, 'hp': float, 'disp': float, 'cyl': "category"}))
df.head()

Now we have our data in we want to regress mpg (miles per gallon) on hp (horsepower) with fixed effects for cyl (cylinders). Now we *could* just pop in a formula like this `'mpg ~ hp + cyl'` because we took the trouble to declare that `cyl` was of datatype category when reading it in from the csv file. This means that statsmodels will treat it as a category and use it as a fixed effect by default.

But when I read that formula I get nervous that `cyl` might not have been processed correctly (ie it could have been read in as a float, which is what it looks like) and it might just be treated as a float (aka a continuous variable) in the regression. Which is not what we want at all. So, to be safe, and make our intentions explicit (even when the data is of type 'category'), it's best to use the syntax `C(cyl)` to ask for a fixed effect.

Here's a regression which does that:

In [ ]:
results_fe = (smf.ols('mpg ~ hp + C(cyl)', data=df)
                 .fit())
print(results_fe.summary())

We can see here that two of the three possible values of `cyl`:

In [ ]:
df['cyl'].unique()

have been added as fixed effects regressors. The way that `+C(cyl)` has been added makes it so that the coefficients given are relative to the coefficient for the intercept. We can turn the intercept off to get a coefficient per unique `cyl` value:

In [ ]:
print(smf.ols('mpg ~ hp + C(cyl) -1', data=df)
         .fit()
         .summary()
         .tables[1])

## Transformations of regressors

This chapter is showcasing *linear* regression. What that means is that the model is linear in the regressors: but it doesn't mean that those regressors can't be some kind of (potentially non-linear) transform of the original features $x_i$.

### Logs and arcsinh

You have two options for adding in logs: do them before, or do them in the formula. Doing them before just makes use of standard dataframe operations to declare a new column:


In [ ]:
df['lnhp'] = np.log(df['hp'])
print(smf.ols('mpg ~ lnhp', data=df)
         .fit()
         .summary()
         .tables[1])

Alternatively, you can specify the log directly in the formula:

In [ ]:
print(smf.ols('mpg ~ np.log(hp)', data=df)
         .fit()
         .summary()
         .tables[1])

Clearly, the first method will work for `arcsinh(x)` and `log(x+1)`, but you can also pass both of these into the formula directly too. (For more on the pros and cons of arcsinh, see {cite}`bellemare2020elasticities`.) Here it is with arcsinh:

In [ ]:
print(smf.ols('mpg ~ np.arcsinh(hp)', data=df)
         .fit()
         .summary()
         .tables[1])

### Interaction terms and powers

This chapter is showcasing *linear* regression. What that means is that the model is linear in the regressors: but it doesn't mean that those regressors can't be some kind of non-linear transform of the original features $x_i$. Two of the most common transformations that you might want to use are *interaction terms* and *polynomial terms*. An example of an interaction term would be

$$
y = \beta_0 + \beta_1 x_1 \cdot x_2
$$

while an example of a polynomial term would be

$$
y = \beta_0 + \beta_1 x_1^2
$$

i.e. the last term enters only after it is multiplied by itself.

One note of warning: the interpretation of the effect of a variable is no longer as simple as was set out at the start of this chapter. To work out *what* the new interpretation is, the procedure is the same though: just take the derivative. In the case of the interaction model above, the effect of a unit change in $x_1$ on $y$ is now going to be a function of $x_2$. In the case of the polynomial model above, the effect of a unit change in $x_1$ on $y$ will be $2\beta_1 \cdot x_1$.

Alright, with all of that preamble out of the way, let's see how we actual do some of this! Let's try including a linear and squared term in the regression of `mpg` on `hp` making use of the numpy power function:

In [ ]:
res_poly = smf.ols('mpg ~ hp + np.power(hp, 2)', data=df)
print(res_poly.fit().summary().tables[1])

Now let's include the original term in hp, a term in disp, and the interaction between them, which is represented by hp:disp in the table.

In [ ]:
res_inter = smf.ols('mpg ~ hp * disp', data=df)
print(res_inter.fit().summary().tables[1])

In the unusual case that you want *only* the interaction term, you write it as it appears in the table above:

In [ ]:
print(smf.ols('mpg ~ hp : disp', data=df).fit().summary().tables[1])

## The formula API explained

As you will have seen `~` separates the left- and right-hand sides of the regression. `+` computes a set union, which will also be familiar from the examples above (ie it inludes two terms as long as they are distinct). `-` computes a set difference; it adds the set of terms to the left of it while removing any that appear on the right of it. As we've seen, `a*b` is a short-hand for `a + b + a:b`, with the last term representing the interaction. `/` is short hand for `a + a:b`, which is useful if, for example `b` is nested within `a`, so it doesn't make sense to control for `b` on its own. Actually, the `:` character can interact multiple terms so that `(a + b):(d + c)` is the same as `a:c + a:d + b:c + b:d`. `C(a)` tells statsmodels to treat `a` as a categorical variable that will be included as a fixed effect. Finally, as we saw above with powers, you can also pass in vectorised functions, such as `np.log` and `np.power`, directly into the formulae.

One gotcha with the formula API is ensuring that you have sensible variable names in your dataframe, i.e. ones that do *not* include whitespace or, to take a really pathological example, have the name 'a + b' for one of the columns that you want to regress on. You can dodge this kind of problem by passing in the variable name as, for example, `Q("a + b")` to be clear that the *column name* is anything within the `Q("...")`.

## Multiple regression models

As is so often the case, you're likely to want to run more than one model at once.

```python

import pandas as pd
from sklearn import datasets
import statsmodels.api as sm
from stargazer.stargazer import Stargazer

diabetes = datasets.load_diabetes()
df = pd.DataFrame(diabetes.data)
df.columns = ['Age', 'Sex', 'BMI', 'ABP', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6']
df['target'] = diabetes.target

est = sm.OLS(endog=df['target'], exog=sm.add_constant(df[df.columns[0:4]])).fit()
est2 = sm.OLS(endog=df['target'], exog=sm.add_constant(df[df.columns[0:6]])).fit()


stargazer = Stargazer([est, est2])
```

## Specifying regressions without formulae, using the array API

As noted, there are two ways to run regressions in [**statsmodels**](https://www.statsmodels.org/stable/index.html); passing the data directly as objects, and using formulae. We've seen the formula API, now let's see how to specify regressions using arrays with the format `sm.OLS(y, X)`.

We will first need to take the data out of the **pandas** dataframe and put it into a couple of arrays. When we're not using the formula API, the default is to treat the array X as the design matrix for the regression-so, if it doesn't have a column of constants in, there will be no intercept in the regression. Therefore, we need to add a constant vector to the matrix `X` if we *do* want an intercept. Use `sm.add_constant(X)` for this.

In [ ]:
X = np.array(xf['height'])
y = np.array(xf['mass'])
X = sm.add_constant(X)
results = sm.OLS(y, X).fit()
print(results.summary())


This approach seems a lot less convenient, not to mention less clear, so you may be wondering when it is useful. It's useful when you want to do many regressions in a systematic way or when you don't know what the columns of a dataset will be called ahead of time. It can actually be a little bit simpler to specify for more complex regressions too.

### Fixed effects in the array API

If you're using the formula API, it's easy to turn a regressor `x` into a fixed effect by putting `C(x)` into the model formula, as you'll see in the next section.

For the array API, things are not that simple and you need to use dummy variables. Let's say we have some data like this:

In [ ]:
from numpy.random import Generator, PCG64

# Set seed for random numbers
seed_for_prng = 78557
prng = Generator(PCG64(seed_for_prng))

no_obs = 200
X = pd.DataFrame(prng.normal(size=no_obs))
X[1] = prng.choice(['a', 'b'], size=no_obs)
# Get this a numpy array
X = X.values
# Create the y data, adding in a bit of noise
y = X[:, 0]*2 + 0.5 + prng.normal(scale=0.1, size=no_obs)
y = [el_y + 1.5 if el_x == 'a' else el_y + 3.4 for el_y, el_x in zip(y, X[:, 1])]
X[:5, :]

The first feature (column) is of numbers and it's clear how we include it. The second, however, is a grouping that we'd like to include as a fixed effect. But if we just throw this matrix into `sm.OLS(y, X)`, we're going to get trouble because **statsmodels** isn't sure what to do with a vector of strings. So, instead, we need to create some dummy variables out of our second column of data

Astonishingly, there are several popular ways to create dummy variables in Python: **scikit-learn**'s `OneHotEncoder` and **pandas**' `get_dummies` being my favourites. Let's use the latter here.

In [ ]:
pd.get_dummies(X[:, 1])

We just need to pop this into our matrix $X$:

In [ ]:
X = np.column_stack([X[:, 0], pd.get_dummies(X[:, 1])])
X = np.array(X, dtype=float)
X[:5, :]

Okay, so now we're ready to do our regression:

In [ ]:
print(sm.OLS(y, X).fit().summary())

Perhaps you can see why I generally prefer the formula API...

## Review

In this very short introduction to regression with code, you should have learned how to:

- [x] ...